<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%EC%97%B0%EC%B2%B4%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/TabNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-tabnet

In [91]:
import pandas as pd
import numpy as np
import torch

from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.metrics import Metric
from sklearn.model_selection import StratifiedKFold ,train_test_split
from sklearn.preprocessing import LabelEncoder


In [56]:
PATH = 'drive/My Drive/dacon/credit'

In [93]:
train = pd.read_csv(PATH+'/dataset/train.csv').drop(['index','FLAG_MOBIL'],axis=1)
test = pd.read_csv(PATH+'/dataset/test.csv').drop(['index','FLAG_MOBIL'],axis=1)

In [103]:
train['occyp_type'].fillna('one two three',inplace=True)
test['occyp_type'].fillna('one two three',inplace=True)
train['begin_months'] = train['begin_month'].values%12
test['begin_months'] = test['begin_month'].values%12
train['not_employed'] = (train['DAYS_EMPLOYED']>0).astype(np.int32)
test['not_employed'] = (test['DAYS_EMPLOYED']>0).astype(np.int32)

In [105]:
train_df,val_df = train_test_split(train,test_size=0.2,stratify=train['credit'])
test_df = test.copy()

le = LabelEncoder()
for col in ['gender','car','reality','work_phone','phone','email']:
    le.fit(train[[col]])
    train_df[[col]] = le.transform(train_df[[col]]).astype(np.int32)
    val_df[[col]] = le.transform(val_df[[col]]).astype(np.int32)
    test_df[[col]] = le.transform(test_df[[col]]).astype(np.int32)

categorical_columns = ['income_type','edu_type','family_type','house_type','occyp_type','begin_months']
categorical_dims = dict()
for col in categorical_columns:
    le.fit(train[[col]])
    train_df[[col]] = le.fit_transform(train_df[[col]])
    val_df[[col]] = le.fit_transform(val_df[[col]])
    test_df[[col]] = le.fit_transform(test_df[[col]])
    categorical_dims[col] = len(le.classes_)

features = train_df.drop('credit',axis=1).columns
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for f in features if f in categorical_columns]

X_train = train_df.drop('credit',axis=1).values
y_train = train_df['credit'].values

X_valid = val_df.drop('credit',axis=1).values
y_valid = val_df['credit'].values

X_test = test_df.values

clf = TabNetClassifier(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=2,
    n_d=8,n_a=8,n_steps=3,gamma=1.3,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params={"swa_lr":0.01},
    scheduler_fn=torch.optim.swa_utils.SWALR,
    mask_type='entmax',
    device_name = 'cuda'
)

clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric=['logloss'],
    patience=50,
    num_workers=0,
    batch_size=1024,
    virtual_batch_size=128,
)



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is 

Device used : cuda
epoch 0  | loss: 0.94476 | val_0_logloss: 1.12451 |  0:00:01s
epoch 1  | loss: 0.87917 | val_0_logloss: 0.88291 |  0:00:02s
epoch 2  | loss: 0.86701 | val_0_logloss: 0.88113 |  0:00:04s
epoch 3  | loss: 0.85164 | val_0_logloss: 0.86952 |  0:00:05s
epoch 4  | loss: 0.83563 | val_0_logloss: 0.8746  |  0:00:07s
epoch 5  | loss: 0.82046 | val_0_logloss: 0.85458 |  0:00:08s
epoch 6  | loss: 0.81561 | val_0_logloss: 0.82897 |  0:00:10s
epoch 7  | loss: 0.81452 | val_0_logloss: 0.81516 |  0:00:11s
epoch 8  | loss: 0.81232 | val_0_logloss: 0.81246 |  0:00:13s
epoch 9  | loss: 0.81064 | val_0_logloss: 0.81154 |  0:00:14s
epoch 10 | loss: 0.81026 | val_0_logloss: 0.80597 |  0:00:15s
epoch 11 | loss: 0.81173 | val_0_logloss: 0.80401 |  0:00:17s
epoch 12 | loss: 0.81061 | val_0_logloss: 0.807   |  0:00:19s
epoch 13 | loss: 0.80894 | val_0_logloss: 0.80374 |  0:00:20s
epoch 14 | loss: 0.80761 | val_0_logloss: 0.80383 |  0:00:22s
epoch 15 | loss: 0.80985 | val_0_logloss: 0.80506 |

In [108]:
submission = pd.read_csv(PATH+'/dataset/sample_submission.csv')
submission.iloc[:,1:] = clf.predict_proba(X_test)

In [109]:
submission.head(20)

,index,0,1,2
0,26457,0.156140,0.213741,0.630119
1,26458,0.083042,0.146327,0.770632
2,26459,0.159687,0.185087,0.655226
3,26460,0.148458,0.208375,0.643166
4,26461,0.139848,0.249127,0.611024
5,26462,0.147482,0.254653,0.597865
6,26463,0.510827,0.329742,0.159431
7,26464,0.123184,0.193046,0.683770
8,26465,0.119793,0.226275,0.653932
9,26466,0.114807,0.238929,0.646265


In [110]:
submission.to_csv(PATH+'/result/tabnet1.csv',index=False)